<a href="https://colab.research.google.com/github/TheKidKy/brain_tumor_detection/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.transforms import v2
from torchvision import datasets, models, transforms
from torchvision.models import ResNet50_Weights, ResNet101_Weights
from torchvision.transforms import ToTensor
import torchvision.utils as vutils
from torch.utils.data import Dataset, DataLoader
import torchvision
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import numpy as np
import os
import copy
import time
from tqdm import tqdm
import cv2 as cv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Check if a GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


### Loading Data

In [4]:
folder = '/content/drive/MyDrive/Brain_Tumor/normalized'
training_directory = folder + '/train'
testing_directory = folder + '/test'
xtrain = []
ytrain = []
dic = {
    "IMAGE": [],
    "RESULT": []
}

try:

    for folder in os.listdir(training_directory):
        folder_path = os.path.join(training_directory, folder)

        if os.path.isdir(folder_path):  # Ensure it's a directory
            for image_filename in os.listdir(folder_path):

                image_path = os.path.join(folder_path, image_filename)

                #
                image = cv.imread(image_path, 0)
                if image is not None:
                    # image=image.reshape(-1)
                    xtrain.append(image)
                    ytrain.append(folder)

            dic["IMAGE"] = xtrain
            dic["RESULT"] = ytrain



            # Clear lists for the next folder


except KeyboardInterrupt or Exception as e:
    print(f"Error: {e}")

In [5]:
df = pd.DataFrame(dic)
print(df.tail())

                                                   IMAGE    RESULT
11693  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  no_tumor
11694  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  no_tumor
11695  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  no_tumor
11696  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  no_tumor
11697  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  no_tumor


In [6]:
label_map = {'no_tumor': 0, 'glioma_tumor': 1, 'meningioma_tumor': 2, 'pituitary_tumor': 3}

X = df.drop("RESULT", axis=1).values   # pixel values
y = df["RESULT"].values                # labels

In [7]:
class TumorDataset(Dataset):
    def __init__(self, df, transform=None):
        self.images = df['IMAGE'].values
        self.labels = df['RESULT'].values
        self.transform = transform

        # map class names to integers
        classes = sorted(df['RESULT'].unique())  # ensure consistent order
        self.label_map = {c: i for i, c in enumerate(classes)}

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = np.array(self.images[idx], dtype=np.float32)  # (H, W)
        img = np.expand_dims(img, axis=-1)                  # (H, W, 1)
        img = np.repeat(img, 3, axis=-1)                    # (H, W, 3)

        if self.transform:
            img = self.transform(img)

        label = self.label_map[self.labels[idx]]
        return img, label

In [8]:
sample_img = np.array(df['IMAGE'].iloc[0])
print(sample_img.shape)

(224, 224)


### Transform To Tensors

In [9]:
train_transform = transforms.Compose([
    transforms.ToTensor(),  # (H,W,C) -> (C,H,W) and scale [0,255] to [0,1]
    transforms.Resize((224,224))
])

In [10]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['RESULT'], random_state=42)

train_dataset = TumorDataset(train_df, transform=train_transform)
val_dataset = TumorDataset(val_df,   transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


dataloaders = {
    'train': train_loader,
    'test': val_loader
}

datasets = {
    'train': train_dataset,
    'test': val_dataset
}

### Initializing ResNet50 Model

In [24]:
def create_resnet50_model(num_classes=4):
    """
    Create ResNet50 model with pretrained ImageNet weights
    """
    # Load pretrained ResNet50
    model = models.resnet50(ResNet50_Weights.IMAGENET1K_V2)

    # Freeze some layers initially
    for param in model.layer3.parameters():
        param.requires_grad = False
    for param in model.layer2.parameters():
        param.requires_grad = False
    for param in model.layer1.parameters():
        param.requires_grad = False


    # Replace the final fully connected layer
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )

    return model

# Create model
model = create_resnet50_model(num_classes=4)
# print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Training Model

In [20]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Only optimize the final layers (unfreeze some layers if needed)
optimizer = optim.Adam([
    {'params': model.fc.parameters(), 'lr': 0.0001},
    {'params': model.layer4.parameters(), 'lr': 0.0001},
    {'params': model.layer3.parameters(), 'lr': 0.0001},  # Adjusting learning rate for layer3
    {'params': model.layer2.parameters(), 'lr': 0.00002},
    {'params': model.layer1.parameters(), 'lr': 0.00002}, # Lower learning rate for earlier layers

])


# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

Using device: cuda


In [21]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, patience=5):
    """
    Train the model with early stopping
    """
    best_acc = -float('inf')  # Initialize to negative infinity for first comparison
    patience_counter = 0  # Counter for early stopping
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects.double() / len(datasets[phase])

            # Store history
            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc.item())
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'best_model.pth')
                patience_counter = 0  # Reset patience if we have improved
            elif phase == 'test' and epoch_acc <= best_acc:
                patience_counter += 1

        # Early stopping condition
        if patience_counter >= patience:
            print(f'Early stopping triggered at epoch {epoch}')
            break

        print()

    print(f'Best val Acc: {best_acc:.4f}')
    return model, history

### Training Execution

In [23]:
num_epochs = 35
model, history = train_model(model, criterion, optimizer, scheduler, num_epochs)

Epoch 0/34
----------


100%|██████████| 293/293 [01:17<00:00,  3.79it/s]


train Loss: 0.0113 Acc: 0.9963


100%|██████████| 74/74 [00:07<00:00,  9.42it/s]


test Loss: 0.3276 Acc: 0.9201

Epoch 1/34
----------


...

test Loss: 0.2713 Acc: 0.9329

Epoch 16/34
----------


100%|██████████| 293/293 [01:18<00:00,  3.71it/s]


train Loss: 0.0019 Acc: 0.9995


100%|██████████| 74/74 [00:08<00:00,  9.03it/s]

test Loss: 0.2607 Acc: 0.9376
Early stopping triggered at epoch 16
Best val Acc: 0.9415
